#Importing Libraries

In [87]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import pandas as pd

#Function for convolution

In [51]:
def conv(image,filter):
  res=np.zeros(image.shape)
  r,c=image.shape
  image=np.insert(image,r,image[r-2,:],axis=0)
  image=np.insert(image,0,image[1,:],axis=0)
  image=np.insert(image,c,image[:,c-2],axis=1)
  image=np.insert(image,0,image[:,1],axis=1)
  r,c=image.shape
  for i in range(1,r-1):
    for j in range(1,c-1):
      res[i-1,j-1]=(image[i-1:i+2,j-1:j+2]*filter).sum()
  return res

#Calibrating the image(All calculations are in m)

In [589]:
def calib(str1,z_0):
    k=0.04
    sobel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])
    img_1=cv2.cvtColor(cv2.imread(str1),cv2.COLOR_BGR2GRAY).astype(np.float32)
    I_x=conv(img_1,sobel_x)
    I_y=conv(img_1,sobel_y)
    Ixx=I_x**2
    Iyy=I_y**2
    Ixy = I_x *I_y
    window_size=3
    Ixx_sum = cv2.GaussianBlur(Ixx, (window_size, window_size), 0)
    Iyy_sum = cv2.GaussianBlur(Iyy, (window_size, window_size), 0)
    Ixy_sum = cv2.GaussianBlur(Ixy, (window_size, window_size), 0)
    det_M = Ixx_sum * Iyy_sum - Ixy_sum ** 2
    trace_M = Ixx_sum + Iyy_sum
    R=det_M-k*trace_M**2
    th=0.0005
    R[R<th*np.max(R)]=0
    corners = np.zeros(R.shape).astype(np.float32)
    r,c=R.shape
    for i in range(1,r-1):
      for j in range(1,c-1):
        if R[i, j] > 0:
                    if (R[i, j] >= R[i - 1, j - 1] and
                        R[i, j] >= R[i - 1, j] and
                        R[i, j] >= R[i - 1, j + 1] and
                        R[i, j] >= R[i, j - 1] and
                        R[i, j] >= R[i, j + 1] and
                        R[i, j] >= R[i + 1, j - 1] and
                        R[i, j] >= R[i + 1, j] and
                        R[i, j] >= R[i + 1, j + 1]):
                        corners[i-1:i+2, j-1:j+2] = 0
                        corners[i,j]=255
    l,m=0,0
    y=np.zeros((8,11,2))
    r,c=corners.shape
    for i in range(0,1024):
        for j in range(0,1024):
          if(corners[i][j]==255):
              y[l,m,0]=i
              m=m+1
              if(m==11):
                l=l+1
                m=0
    l=0
    m=0
    for j in range(0,1024):
        for i in range(0,1024):
          if(corners[i][j]==255):
              y[l,m,1]=j
              l=l+1
              if(l==8):
                m=m+1
                l=0
    l=0
    m=0
    X=np.array([[0,0,z_0/1000,1]])
    z=np.zeros((1,4))
    l1=np.hstack((z,-X,y[0,0,1]*X))
    l2=np.hstack((X,z,-y[0,0,0]*X))
    a= np.vstack((l1,l2))
    for i in range(8):
      for j in range(11):
        if(i!=0 or j!=0):
          X=np.array([[i*0.025,j*0.025,z_0/1000,1]])
          z=np.zeros((1,4))
          l1=np.hstack((z,-X,y[i,j,1]*X))
          l2=np.hstack((X,z,-y[i,j,0]*X))
          a= np.vstack((a,l2))
          a= np.vstack((a,l1))
    U, S, Vt = np.linalg.svd(a, full_matrices=False)
    rank = np.linalg.matrix_rank(a)
    eigenvector_min = Vt[rank, :]
    P=eigenvector_min.reshape((3,4))
    print("Calibration Matrix:")
    print(P/P[-1,-1])
    return P

In [500]:
z=pd.read_csv('z-dimension.csv')

#Calibration and Intrinsic Camera Matrix for Image 1

In [600]:
P=calib('c1.png',z.iloc[0,1])
X=P
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[ 2.18620173e-10  5.81651715e-13 -3.07632119e-02  1.23052848e-02]
 [ 6.13619027e-12  2.13359769e-10  2.19207524e-04 -8.76830067e-05]
 [ 4.26109485e-15  1.14900113e-15 -2.50000000e+00  1.00000000e+00]]
[[2.18513043e-10 6.86629162e-12 3.07632119e-02]
 [0.00000000e+00 2.13447989e-10 2.13447989e-10]
 [0.00000000e+00 0.00000000e+00 2.50000000e+00]]


#Calibration and Intrinsic Camera Matrix for Image 2

In [591]:
P=calib('c2.png',z.iloc[1,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[ 3.41927770e-11  5.94866589e-14  3.37773095e-03 -1.68886547e-03]
 [ 8.09756418e-13  3.36363214e-11 -1.25973638e-05  6.29868356e-06]
 [ 9.44047107e-16  4.15342693e-16 -2.00000000e+00  1.00000000e+00]]
[[ 3.41814430e-11  8.82383454e-13 -3.37773095e-03]
 [ 0.00000000e+00  3.36460670e-11  3.36460670e-11]
 [ 0.00000000e+00  0.00000000e+00  2.00000000e+00]]


#Calibration and Intrinsic Camera Matrix for Image 3

In [592]:
P=calib('c3.png',z.iloc[2,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[-9.09954441e-11 -7.67133048e-13 -8.35626107e-04  5.01375652e-04]
 [ 2.99433028e-12 -9.42577228e-11  3.25741827e-04 -1.95445105e-04]
 [ 3.26967069e-16 -1.05232970e-15 -1.66666667e+00  1.00000000e+00]]
[[ 9.09739215e-11 -2.12249221e-12  8.35626107e-04]
 [ 0.00000000e+00  9.43052722e-11  9.43052719e-11]
 [ 0.00000000e+00  0.00000000e+00  1.66666667e+00]]


#Calibration and Intrinsic Camera Matrix for Image 4

In [593]:
P=calib('c4.png',z.iloc[3,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[-7.58141363e-11 -6.30373410e-13  1.13173415e-04 -7.92214020e-05]
 [-4.70766374e-12 -7.08269141e-11  6.39195637e-04 -4.47436955e-04]
 [-2.79673472e-15  4.46424207e-16 -1.42857143e+00  1.00000000e+00]]
[[ 7.56054134e-11  5.65704268e-12 -1.13173415e-04]
 [ 0.00000000e+00  7.09831941e-11  7.09831942e-11]
 [ 0.00000000e+00  0.00000000e+00  1.42857143e+00]]


#Calibration and Intrinsic Camera Matrix for Image 5

In [594]:
P=calib('c5.png',z.iloc[4,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[ 1.89867917e-11  4.82393870e-13 -4.90895995e-04  3.92716800e-04]
 [ 2.13988451e-13  1.98105834e-11 -7.64128425e-04  6.11302743e-04]
 [ 8.15186976e-16  1.85153245e-16 -1.25000000e+00  1.00000000e+00]]
[[1.89804735e-11 6.87443371e-13 4.90895995e-04]
 [0.00000000e+00 1.98117390e-11 1.98117391e-11]
 [0.00000000e+00 0.00000000e+00 1.25000000e+00]]


#Calibration and Intrinsic Camera Matrix for Image 6

In [595]:
P=calib('c6.png',z.iloc[5,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[ 1.04822843e-11 -6.44832391e-13  1.80352309e-04 -1.62317076e-04]
 [ 3.95208180e-12  8.82684128e-12  1.69315627e-04 -1.52384062e-04]
 [ 2.15270181e-15  1.48041199e-16 -1.11111111e+00  1.00000000e+00]]
[[ 9.83062170e-12  3.69499348e-12 -1.80352309e-04]
 [ 0.00000000e+00  9.67119851e-12  9.67119835e-12]
 [ 0.00000000e+00  0.00000000e+00  1.11111111e+00]]


#Calibration and Intrinsic Camera Matrix for Image 7

In [596]:
P=calib('c7.png',z.iloc[6,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[-1.52295159e-11 -3.31350915e-13 -7.40426418e-04  7.40426414e-04]
 [ 2.21776010e-14 -1.42322002e-11  3.00409912e-05 -3.00409946e-05]
 [-1.83615170e-15 -5.83092011e-16 -1.00000000e+00  1.00000000e+00]]
[[1.52300124e-11 3.07618913e-13 7.40426418e-04]
 [0.00000000e+00 1.42322175e-11 1.42322175e-11]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


#Calibration and Intrinsic Camera Matrix for Image 8

In [597]:
P=calib('c8.png',z.iloc[7,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print(K)

Calibration Matrix:
[[-5.88310077e-11 -2.47922705e-12  2.70826361e-03 -2.97908999e-03]
 [-5.73718025e-12 -5.24704723e-11  1.74454043e-03 -1.91899449e-03]
 [-8.80394516e-15 -2.18272200e-15 -9.09090909e-01  1.00000000e+00]]
[[ 5.82130007e-11  8.85909347e-12 -2.70826361e-03]
 [ 0.00000000e+00  5.27832013e-11  5.27831953e-11]
 [ 0.00000000e+00  0.00000000e+00  9.09090909e-01]]


#Calibration and Intrinsic Camera Matrix for Image 9

In [598]:
P=calib('c9.png',z.iloc[8,1])
P=P/P[-1,-1]
P=P[:,:3]
m1,m2,m3=P[0,:],P[1],P[2]
r3=m3/np.linalg.norm(m3)
e2=m2-(np.dot(m2.T,r3.T)*r3)
r2=e2/np.linalg.norm(e2)
e1=m1-(np.dot(m1.T,r3.T)*r3)-(np.dot(m1.T,r2.T)*r2)
r1=e1/np.linalg.norm(e1)
K=np.zeros((3,3))
K[0,0]=np.dot(e1.T,r1.T)
K[0,1]=np.dot(m1.T,r2.T)
K[0,2]=np.dot(m1.T,r3.T)
K[1,1]=np.dot(e2.T,r2.T)
K[1,2]=np.dot(m2.T,r2.T)
K[2,2]=np.dot(m3.T,r3.T)
print("K- Matrix is :")
print(K)

Calibration Matrix:
[[-1.68504871e-11 -3.06310303e-12 -2.58003546e-04  3.09604251e-04]
 [-2.43879913e-12 -1.56287443e-11  1.66267427e-04 -1.99520916e-04]
 [-4.98984797e-15 -3.62698693e-15 -8.33333333e-01  1.00000000e+00]]
K- Matrix is :
[[1.61767319e-11 5.62448383e-12 2.58003546e-04]
 [0.00000000e+00 1.58178829e-11 1.58178820e-11]
 [0.00000000e+00 0.00000000e+00 8.33333333e-01]]
